In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# sklearn 1.0 버전 부터 boston 주택가격 데이터셋이 삭제되었다.
# 다음과 같이 boston 데이터셋을 불러온다.
from sklearn.datasets import fetch_openml
boston = fetch_openml(name='boston')

print(boston.DESCR)
# 506개의 샘플을 가지고 있으며, 13개의 속성들과 이에 대한 타깃 값을 갖고 있다.

In [4]:
# 간단한 탐험적 데이터 분석(exploratory data analysis)을 위해 판다스(Pandas) 데이터 프레임으로 변환 후, 데이터 일부를 확인한다.
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df["TARGET"] = boston.target
df.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TARGET
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0
505,0.04741,0.0,11.93,0,0.573,6.030,80.8,2.5050,1,273.0,21.0,396.90,7.88,11.9


In [ ]:
# 각 속성의 분포와 속성 사이의 선형적 관계 유무 파악을 위해 페어 플롯(pair plot)을 그린다.
sns.pairplot(df)
plt.show()

In [ ]:
# TARGET 속성에 대응되는 맨 마지막 줄을 살펴보면, 일부 속성들이 TARGET 속성과 약간의 선형적 관계를 띄는 것을 볼 수 있다.
# 선형적 관계를 띄는 것으로 보이는 일부 속성을 추려 다시 페어 플롯을 그린다.
cols = ["TARGET", "INDUS", 'RM', "LSTAT", "NOX", "DIS"]
sns.pairplot(df[cols])
plt.show()

In [11]:
# 학습 코드 구현
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 넘파이 데이터를 파이토치 실수형 텐서로 변환한다.
data = torch.from_numpy(df[cols].values).float()
print(data.shape) # torch.Size([506, 6])

# 위의 데이터를 입력 x와 출력 y로 나눈다.
y = data[:, :1]
x = data[:, 1:]
print(x.shape, y.shape) # torch.Size([506, 5]) torch.Size([506, 1])

# 학습에 필요한 설정값
n_epochs = 2000
learning_rate = 1e-3
print_interval = 100

# 모델을 생성한다.
# 텐서 x의 마지막 차원의 크기를 선형 계층의 입력 크기로 주고, 텐서 y의 마지막 차원의 크기를 선형 계층의 출력 크기로준다.
model = nn.Linear(x.size(-1), y.size(-1))
print(model) # Linear(in_features=5, out_features=1, bias=True)

# 옵티마이저를 생성한다.
# 파이토치에서 제공하는 옵티마이저 클래스를 통해 경사하강법을 구현할 수 있다.
# backward 함수를 호출한 후, 옵티마이저 객체에서 step 함수를 호출하면 경사하강을 1번 수행한다.
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

# 학습할 준비는 끝
# 정해진 에포크(epoch)만큼 for 반복문을 통해 최적화를 수행한다.
for i in range(n_epochs):
    y_hat = model(x)
    loss = F.mse_loss(y_hat, y)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    if (i + 1) % print_interval == 0:
        print("Epoch %d: loss=%.4e" % (i + 1, loss))


torch.Size([506, 6])
torch.Size([506, 5]) torch.Size([506, 1])
Linear(in_features=5, out_features=1, bias=True)
Epoch 100: loss=4.6113e+01
Epoch 200: loss=3.8596e+01
Epoch 300: loss=3.4416e+01
Epoch 400: loss=3.2060e+01
Epoch 500: loss=3.0732e+01
Epoch 600: loss=2.9983e+01
Epoch 700: loss=2.9560e+01
Epoch 800: loss=2.9322e+01
Epoch 900: loss=2.9187e+01
Epoch 1000: loss=2.9111e+01
Epoch 1100: loss=2.9068e+01
Epoch 1200: loss=2.9043e+01
Epoch 1300: loss=2.9029e+01
Epoch 1400: loss=2.9021e+01
Epoch 1500: loss=2.9016e+01
Epoch 1600: loss=2.9012e+01
Epoch 1700: loss=2.9010e+01
Epoch 1800: loss=2.9009e+01
Epoch 1900: loss=2.9008e+01
Epoch 2000: loss=2.9007e+01


In [ ]:
# 결과 확인
# 학습이 잘 수행되었는지 확인한다.
# 29로 수렴하는 손실값을 통해서도 학습이 진행된 정도를 어느정도 파악할 수 있다.
# 하지만, 해당 값만 보고 직관적으로 파악할 수 없기에, 시각화를 통해 다시 한번 확인한다.
# 마지막으로 모델을 통과(feed-forward)한 y_hat을 가져와서 실제 y와 비교하기 위한 페어 플롯을 그린다.
df = pd.DataFrame(torch.cat([y, y_hat], dim=1).detach_().numpy(), columns = ["y", "y_hat"])
sns.pairplot(df, height=5)
plt.show()
# 왼쪽 위에 그려진 y의 분포와 오른쪽 아래에 그려진 y_hat의 분포가 약간은 다르게 나온 것을 볼 수 있다.
# 하지만, 오른쪽 위에 그려진 y와 왼쪽 아래의 y_hat과의 비교에선, 대부분의 점들이 빨간색 점선 부근에 나타나 있는 것을 확인할 수 있다.